In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.text import Tokenizer
from torchtext import data
from torchtext.data import TabularDataset
from tqdm.notebook import tqdm

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
chart = train[['text','target']]
t_chart = test[['text']]

In [4]:
def remove(text):
    text = text.str.lower()
    text = text.str.replace('http\S+','')# http://와 https:// 두 가지 버전 있음
    text = text.str.replace('[0-9]','')# 사람 숫자, 날짜
    text = text.str.replace('@\S+','')# '@' 뒤에는 아이디 태그
    text = text.str.replace(',','')
    return text

In [5]:
chart["text"] = remove(chart["text"])
t_chart["text"] = remove(t_chart["text"])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
chart.to_csv('train_r.csv',index = False)
t_chart.to_csv('test_r.csv',index = False)

In [7]:
pd.read_csv('train_r.csv').target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
len(pd.read_csv('train_r.csv'))

7613

# CSV -> trainset (= TabularDataset)

In [9]:
BATCH_SIZE = 64
lr = 0.001
epochs = 20
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [10]:
TEXT = data.Field(sequential = True, tokenize = str.split, batch_first = True)
LABEL = data.LabelField(dtype = torch.long)

In [11]:
trainset = TabularDataset(path = 'train_r.csv', format='csv', fields=[('text', TEXT), ('label', LABEL)])
testset = TabularDataset(path = 'test_r.csv', format='csv', fields=[('text', TEXT)])

In [12]:
trainset.examples[0].text, trainset.examples[0].label

(['text'], 'target')

In [13]:
trainset.examples.pop(0) # column을 제거해야 합니다.

In [14]:
trainset.examples[0].text, trainset.examples[0].label

(['our',
  'deeds',
  'are',
  'the',
  'reason',
  'of',
  'this',
  '#earthquake',
  'may',
  'allah',
  'forgive',
  'us',
  'all'],
 '1')

# trainset -> train_iter (= torchtext.data.Iterator)

In [15]:
TEXT.build_vocab(trainset, min_freq = 2)# 최소 2번 이상 등장한 단어만 사전에 담음
LABEL.build_vocab(trainset)

In [16]:
trainset, valset = trainset.split(split_ratio = 0.9)

In [17]:
train_iter = data.Iterator(dataset = trainset, batch_size = BATCH_SIZE, device = DEVICE)
val_iter = data.Iterator(dataset = valset, batch_size = BATCH_SIZE, device = DEVICE)
test_iter = data.Iterator(dataset = testset, batch_size = BATCH_SIZE, device = DEVICE)

In [18]:
vocab_size = len(TEXT.vocab)
n_classes = 2

# model

In [19]:
class LSTM_model(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embedding_dim, n_classes):
        super(LSTM_model, self).__init__()
        self.n_layers = n_layers
        self.embed = nn.Embedding(n_vocab, embedding_dim)
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(embedding_dim, self.hidden_dim, num_layers = self.n_layers, batch_first = True)
        self.out = nn.Linear(self.hidden_dim, n_classes)
        
    def forward(self, x):
        # x = [64, 27]
        x = self.embed(x)
        # x = [64, 27, 128]
        h_0 = self._init_state(batch_size = x.size(0))#첫 번째 은닉 벡터 정의
        # h_0 = [1, 64, 256]
        x, _ = self.lstm(x,(h_0,h_0))
        # x = [64, 27, 256]
        h_t = x[:,-1,:]
        # h_t = [64, 256]
        logit = self.out(h_t)
        # logit = [64, 2]
        return logit
    
    def _init_state(self, batch_size = 1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

# Train model

In [20]:
def train(model, optimizer, train_iter):
    model.train()
    acc, total_loss = 0, 0
    for b,batch in tqdm(enumerate(train_iter)):
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        optimizer.zero_grad()# 기울기 0으로 초기화
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction = 'mean')
        total_loss += loss.item()
        acc += (logit.max(1)[1].view(y.size()).data == y.data).sum()
        loss.backward()
        optimizer.step()
    size = len(train_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100 * acc / size
    return avg_loss, avg_accuracy

# Evaluate model

In [21]:
def evaluate(model, val_iter):
    model.eval()
    acc, total_loss = 0, 0
    for batch in val_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction = 'sum')#오차의 합 구하고 total_loss에 더해줌
        total_loss += loss.item()
        acc += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100 * acc / size
    return avg_loss, avg_accuracy

In [22]:
from tqdm import tqdm

# Train

In [23]:
model = LSTM_model(1, 128, vocab_size, 64, n_classes).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

In [24]:
model2 = LSTM_model(1, 128, vocab_size, 128, n_classes).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

In [25]:
model3 = LSTM_model(1, 128, vocab_size, 256, n_classes).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

In [26]:
from time import sleep

In [27]:
best_val_loss = None
for e in tqdm(range(1, epochs + 1)):
    train_loss, train_accuracy = train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)

    print("<<e : %d>> <<train_loss : %5.2f>> <<train_accuracy : %5.2f>> <<val_loss : %5.2f>> <<val_accuracy : %5.2f>>"%(e, train_loss, train_accuracy, val_loss, val_accuracy))
    sleep(0.01)

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:00,  9.59it/s]
3it [00:00, 10.27it/s]
5it [00:00, 10.55it/s]
7it [00:00, 11.10it/s]
9it [00:00, 11.38it/s]
11it [00:00, 11.85it/s]
13it [00:01, 11.84it/s]
15it [00:01, 11.58it/s]
17it [00:01, 11.79it/s]
19it [00:01, 11.67it/s]
21it [00:01, 11.77it/s]
23it [00:01, 11.54it/s]
25it [00:02, 11.57it/s]
27it [00:02, 11.91it/s]
29it [00:02, 12.10it/s]
31it [00:02, 12.23it/s]
33it [00:02, 12.46it/s]
35it [00:02, 12.10it/s]
37it [00:03, 11.97it/s]
39it [00:03, 12.07it/s]
41it [00:03, 11.92it/s]
43it [00:03, 11.78it/s]
45it [00:03, 11.70it/s]
47it [00:03, 11.41it/s]
49it [00:04, 10.97it/s]
51it [00:04, 11.10it/s]
53it [00:04, 10.80it/s]
55it [00:04, 11.20it/s]
57it [00:04, 11.82it/s]
59it [00:05, 12.03it/s]
61it [00:05, 11.87it/s]
63it [00:05, 11.66it/s]
65it [00:05, 11.79it/s]
67it [00:05, 11.88it/s]
69it [00:05, 12.00it/s]
71it [00:06, 11.90it/s]
73

<<e : 1>> <<train_loss :  0.01>> <<train_accuracy : 56.00>> <<val_loss :  0.68>> <<val_accuracy : 55.00>>


  5%|████▏                                                                              | 1/20 [00:09<03:00,  9.49s/it]
0it [00:00, ?it/s]
1it [00:00,  9.87it/s]
3it [00:00, 10.55it/s]
5it [00:00, 10.90it/s]
7it [00:00, 11.12it/s]
8it [00:00, 10.61it/s]
10it [00:00, 11.01it/s]
12it [00:01, 10.98it/s]
14it [00:01, 11.18it/s]
16it [00:01, 11.22it/s]
18it [00:01, 11.43it/s]
20it [00:01, 11.70it/s]
22it [00:01, 11.94it/s]
24it [00:02, 11.96it/s]
26it [00:02, 12.29it/s]
28it [00:02, 12.23it/s]
30it [00:02, 11.69it/s]
32it [00:02, 11.68it/s]
34it [00:02, 11.76it/s]
36it [00:03, 11.79it/s]
38it [00:03, 11.99it/s]
40it [00:03, 12.06it/s]
42it [00:03, 12.23it/s]
44it [00:03, 12.16it/s]
46it [00:03, 12.01it/s]
48it [00:04, 11.82it/s]
50it [00:04, 11.75it/s]
52it [00:04, 11.87it/s]
54it [00:04, 12.15it/s]
56it [00:04, 12.38it/s]
58it [00:04, 12.18it/s]
60it [00:05, 11.83it/s]
62it [00:05, 11.78it/s]
64it [00:05, 11.56it/s]
66it [00:05, 11.49it/s]
68it [00:05, 11.31it/s]
70it [00:05, 11.32it/s]
72

<<e : 2>> <<train_loss :  0.01>> <<train_accuracy : 57.00>> <<val_loss :  0.69>> <<val_accuracy : 55.00>>


 10%|████████▎                                                                          | 2/20 [00:18<02:50,  9.49s/it]
0it [00:00, ?it/s]
1it [00:00,  9.78it/s]
3it [00:00, 10.20it/s]
5it [00:00, 10.61it/s]
7it [00:00, 11.05it/s]
9it [00:00, 11.58it/s]
11it [00:00, 11.43it/s]
13it [00:01, 11.54it/s]
15it [00:01, 11.82it/s]
17it [00:01, 11.84it/s]
19it [00:01, 12.20it/s]
21it [00:01, 12.39it/s]
23it [00:01, 12.65it/s]
25it [00:02, 12.48it/s]
27it [00:02, 12.40it/s]
29it [00:02, 12.06it/s]
31it [00:02, 12.14it/s]
33it [00:02, 12.21it/s]
35it [00:02, 12.36it/s]
37it [00:03, 12.10it/s]
39it [00:03, 11.84it/s]
41it [00:03, 12.00it/s]
43it [00:03, 12.00it/s]
45it [00:03, 12.12it/s]
47it [00:03, 12.09it/s]
49it [00:04, 12.04it/s]
51it [00:04, 12.44it/s]
53it [00:04, 12.57it/s]
55it [00:04, 12.49it/s]
57it [00:04, 12.48it/s]
59it [00:04, 12.38it/s]
61it [00:05, 12.41it/s]
63it [00:05, 12.31it/s]
65it [00:05, 12.20it/s]
67it [00:05, 12.21it/s]
69it [00:05, 12.26it/s]
71it [00:05, 12.17it/s]
73

...

 90%|█████████████████████████████████████████████████████████████████████████▊        | 18/20 [03:03<00:21, 10.59s/it]
0it [00:00, ?it/s]
2it [00:00, 11.27it/s]
4it [00:00, 11.40it/s]
6it [00:00, 11.29it/s]
7it [00:00, 10.82it/s]
9it [00:00, 11.04it/s]
11it [00:00, 11.02it/s]
13it [00:01, 11.02it/s]
15it [00:01, 11.13it/s]
17it [00:01, 11.10it/s]
19it [00:01, 11.22it/s]
21it [00:01, 11.24it/s]
23it [00:02, 11.44it/s]
25it [00:02, 11.69it/s]
27it [00:02, 11.48it/s]
29it [00:02, 11.11it/s]
31it [00:02, 11.16it/s]
33it [00:02, 11.21it/s]
35it [00:03, 11.04it/s]
37it [00:03, 11.22it/s]
39it [00:03, 10.89it/s]
41it [00:03, 10.91it/s]
43it [00:03, 11.07it/s]
45it [00:04, 11.09it/s]
47it [00:04, 10.64it/s]
49it [00:04, 10.77it/s]
51it [00:04, 10.83it/s]
53it [00:04, 11.12it/s]
55it [00:04, 11.13it/s]
57it [00:05, 10.86it/s]
59it [00:05, 10.71it/s]
61it [00:05, 10.87it/s]
63it [00:05, 10.74it/s]
65it [00:05, 10.79it/s]
67it [00:06, 10.86it/s]
69it [00:06, 10.75it/s]
71it [00:06, 10.81it/s]
73

<<e : 19>> <<train_loss :  0.00>> <<train_accuracy : 97.00>> <<val_loss :  1.26>> <<val_accuracy : 75.00>>


 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [03:13<00:10, 10.50s/it]
0it [00:00, ?it/s]
1it [00:00,  9.87it/s]
3it [00:00, 10.15it/s]
5it [00:00, 10.38it/s]
7it [00:00, 10.43it/s]
9it [00:00, 10.72it/s]
11it [00:01, 10.74it/s]
12it [00:01, 10.33it/s]
14it [00:01, 10.61it/s]
16it [00:01, 10.91it/s]
18it [00:01, 10.91it/s]
20it [00:01, 10.76it/s]
22it [00:02, 10.72it/s]
24it [00:02, 10.95it/s]
26it [00:02, 10.72it/s]
28it [00:02, 10.76it/s]
30it [00:02, 10.59it/s]
32it [00:02, 10.60it/s]
34it [00:03, 10.32it/s]
36it [00:03, 10.61it/s]
38it [00:03, 10.57it/s]
40it [00:03, 10.55it/s]
42it [00:03, 10.40it/s]
44it [00:04, 10.50it/s]
46it [00:04, 10.43it/s]
48it [00:04, 10.47it/s]
50it [00:04, 10.33it/s]
52it [00:04, 10.33it/s]
54it [00:05, 10.41it/s]
56it [00:05, 10.36it/s]
58it [00:05, 10.16it/s]
60it [00:05, 10.18it/s]
62it [00:05, 10.18it/s]
64it [00:06, 10.40it/s]
66it [00:06, 10.41it/s]
68it [00:06, 10.29it/s]
70it [00:06, 10.39it/s]
72

<<e : 20>> <<train_loss :  0.00>> <<train_accuracy : 97.00>> <<val_loss :  1.23>> <<val_accuracy : 75.00>>


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [03:24<00:00, 10.21s/it]


In [28]:
best_val_loss = None
for e in tqdm(range(1, epochs + 1)):
    train_loss, train_accuracy = train(model2, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model2, val_iter)

    print("<<e : %d>> <<train_loss : %5.2f>> <<train_accuracy : %5.2f>> <<val_loss : %5.2f>> <<val_accuracy : %5.2f>>"%(e, train_loss, train_accuracy, val_loss, val_accuracy))
    sleep(0.01)

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:00,  7.33it/s]
2it [00:00,  6.99it/s]
3it [00:00,  6.24it/s]
4it [00:00,  6.69it/s]
5it [00:00,  6.94it/s]
6it [00:00,  7.38it/s]
7it [00:01,  7.23it/s]
8it [00:01,  7.55it/s]
9it [00:01,  7.64it/s]
10it [00:01,  8.06it/s]
11it [00:01,  8.09it/s]
12it [00:01,  8.28it/s]
13it [00:01,  7.93it/s]
14it [00:01,  7.93it/s]
15it [00:01,  7.90it/s]
16it [00:02,  8.08it/s]
17it [00:02,  7.93it/s]
18it [00:02,  7.93it/s]
19it [00:02,  7.96it/s]
20it [00:02,  6.70it/s]
21it [00:02,  6.91it/s]
22it [00:03,  5.11it/s]
23it [00:03,  5.71it/s]
24it [00:03,  5.85it/s]
25it [00:03,  6.52it/s]
26it [00:03,  6.94it/s]
27it [00:03,  7.20it/s]
28it [00:03,  7.48it/s]
29it [00:04,  7.85it/s]
30it [00:04,  8.07it/s]
31it [00:04,  8.14it/s]
32it [00:04,  8.45it/s]
33it [00:04,  8.40it/s]
34it [00:04,  8.34it/s]
35it [00:04,  8.40it/s]
36it [00:04,  8.35it/s]
37it [

<<e : 1>> <<train_loss :  0.01>> <<train_accuracy : 42.00>> <<val_loss :  0.70>> <<val_accuracy : 44.00>>


  5%|████▏                                                                              | 1/20 [00:13<04:11, 13.25s/it]
0it [00:00, ?it/s]
1it [00:00,  9.36it/s]
2it [00:00,  9.39it/s]
3it [00:00,  9.15it/s]
5it [00:00,  9.39it/s]
6it [00:00,  9.42it/s]
7it [00:00,  9.47it/s]
9it [00:00, 10.06it/s]
10it [00:01,  9.83it/s]
12it [00:01,  9.95it/s]
14it [00:01, 10.23it/s]
16it [00:01, 10.39it/s]
18it [00:01, 10.59it/s]
20it [00:01, 10.49it/s]
22it [00:02, 10.47it/s]
24it [00:02, 10.23it/s]
26it [00:02, 10.48it/s]
28it [00:02, 10.46it/s]
30it [00:02, 10.73it/s]
32it [00:03, 10.55it/s]
34it [00:03, 10.53it/s]
36it [00:03, 10.59it/s]
38it [00:03, 10.96it/s]
40it [00:03, 11.14it/s]
42it [00:03, 11.21it/s]
44it [00:04, 10.99it/s]
46it [00:04, 10.73it/s]
48it [00:04, 10.80it/s]
50it [00:04, 10.69it/s]
52it [00:04, 10.77it/s]
54it [00:05, 10.42it/s]
56it [00:05, 10.86it/s]
58it [00:05, 10.80it/s]
60it [00:05, 10.67it/s]
62it [00:05, 10.68it/s]
64it [00:06, 10.63it/s]
66it [00:06, 10.64it/s]
68it

<<e : 2>> <<train_loss :  0.01>> <<train_accuracy : 42.00>> <<val_loss :  0.70>> <<val_accuracy : 44.00>>


 10%|████████▎                                                                          | 2/20 [00:23<03:44, 12.49s/it]
0it [00:00, ?it/s]
2it [00:00, 11.01it/s]
4it [00:00, 11.29it/s]
5it [00:00, 10.82it/s]
7it [00:00, 10.86it/s]
9it [00:00, 10.69it/s]
11it [00:01, 10.82it/s]
13it [00:01, 10.65it/s]
15it [00:01, 10.67it/s]
17it [00:01, 10.53it/s]
19it [00:01, 10.22it/s]
20it [00:01, 10.14it/s]
21it [00:02,  9.87it/s]
22it [00:02,  9.81it/s]
24it [00:02, 10.05it/s]
26it [00:02,  9.93it/s]
27it [00:02,  9.84it/s]
28it [00:02,  9.65it/s]
30it [00:02, 10.09it/s]
32it [00:03, 10.29it/s]
34it [00:03, 10.21it/s]
36it [00:03, 10.27it/s]
38it [00:03, 10.27it/s]
40it [00:03, 10.48it/s]
42it [00:04, 10.46it/s]
44it [00:04, 10.01it/s]
46it [00:04,  9.76it/s]
47it [00:04,  9.43it/s]
49it [00:04,  9.84it/s]
51it [00:04, 10.31it/s]
53it [00:05, 10.40it/s]
55it [00:05, 10.40it/s]
57it [00:05, 10.46it/s]
59it [00:05, 10.54it/s]
61it [00:05, 10.38it/s]
63it [00:06, 10.33it/s]
65it [00:06, 10.43it/s]
67

...

 90%|█████████████████████████████████████████████████████████████████████████▊        | 18/20 [03:46<00:25, 12.74s/it]
0it [00:00, ?it/s]
1it [00:00,  8.18it/s]
2it [00:00,  8.44it/s]
3it [00:00,  8.69it/s]
4it [00:00,  9.01it/s]
5it [00:00,  9.26it/s]
7it [00:00,  9.35it/s]
8it [00:00,  9.06it/s]
9it [00:00,  8.78it/s]
10it [00:01,  7.97it/s]
11it [00:01,  8.07it/s]
12it [00:01,  8.12it/s]
13it [00:01,  8.30it/s]
14it [00:01,  8.52it/s]
15it [00:01,  8.47it/s]
16it [00:01,  8.56it/s]
17it [00:01,  8.66it/s]
18it [00:02,  8.71it/s]
19it [00:02,  9.00it/s]
20it [00:02,  8.78it/s]
21it [00:02,  8.59it/s]
22it [00:02,  8.62it/s]
23it [00:02,  8.59it/s]
24it [00:02,  8.55it/s]
25it [00:02,  8.58it/s]
26it [00:03,  8.46it/s]
27it [00:03,  8.27it/s]
28it [00:03,  8.56it/s]
29it [00:03,  8.77it/s]
30it [00:03,  8.65it/s]
31it [00:03,  8.84it/s]
32it [00:03,  8.33it/s]
33it [00:03,  8.49it/s]
34it [00:03,  8.35it/s]
35it [00:04,  8.26it/s]
36it [00:04,  7.90it/s]
37it [00:04,  7.98it/s]
38it 

<<e : 19>> <<train_loss :  0.01>> <<train_accuracy : 42.00>> <<val_loss :  0.70>> <<val_accuracy : 44.00>>


 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [03:59<00:12, 12.87s/it]
0it [00:00, ?it/s]
1it [00:00,  8.45it/s]
2it [00:00,  8.47it/s]
3it [00:00,  8.38it/s]
4it [00:00,  8.58it/s]
5it [00:00,  8.76it/s]
6it [00:00,  8.95it/s]
7it [00:00,  9.16it/s]
8it [00:00,  9.39it/s]
9it [00:00,  9.29it/s]
10it [00:01,  9.00it/s]
11it [00:01,  9.04it/s]
12it [00:01,  8.83it/s]
13it [00:01,  8.85it/s]
14it [00:01,  8.92it/s]
15it [00:01,  9.16it/s]
16it [00:01,  9.13it/s]
17it [00:01,  9.21it/s]
18it [00:01,  9.38it/s]
19it [00:02,  9.55it/s]
21it [00:02,  9.59it/s]
22it [00:02,  9.22it/s]
23it [00:02,  9.30it/s]
24it [00:02,  9.38it/s]
25it [00:02,  9.16it/s]
26it [00:02,  9.13it/s]
27it [00:02,  9.04it/s]
28it [00:03,  8.83it/s]
29it [00:03,  8.88it/s]
30it [00:03,  8.93it/s]
31it [00:03,  8.85it/s]
32it [00:03,  8.80it/s]
33it [00:03,  8.88it/s]
34it [00:03,  9.01it/s]
35it [00:03,  9.10it/s]
36it [00:03,  9.21it/s]
37it [00:04,  9.35it/s]
38it [

<<e : 20>> <<train_loss :  0.01>> <<train_accuracy : 42.00>> <<val_loss :  0.70>> <<val_accuracy : 44.00>>


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [04:11<00:00, 12.58s/it]


In [29]:
best_val_loss = None
for e in tqdm(range(1, epochs + 1)):
    train_loss, train_accuracy = train(model3, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model3, val_iter)

    print("<<e : %d>> <<train_loss : %5.2f>> <<train_accuracy : %5.2f>> <<val_loss : %5.2f>> <<val_accuracy : %5.2f>>"%(e, train_loss, train_accuracy, val_loss, val_accuracy))
    sleep(0.01)

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:00,  7.98it/s]
2it [00:00,  7.74it/s]
3it [00:00,  6.55it/s]
4it [00:00,  7.04it/s]
5it [00:00,  7.20it/s]
6it [00:00,  7.48it/s]
7it [00:00,  7.71it/s]
8it [00:01,  7.70it/s]
9it [00:01,  7.67it/s]
10it [00:01,  7.71it/s]
11it [00:01,  7.25it/s]
12it [00:01,  7.69it/s]
13it [00:01,  7.67it/s]
14it [00:01,  7.83it/s]
15it [00:02,  7.77it/s]
16it [00:02,  7.51it/s]
17it [00:02,  7.64it/s]
18it [00:02,  7.96it/s]
19it [00:02,  7.84it/s]
20it [00:02,  7.88it/s]
21it [00:02,  7.87it/s]
22it [00:02,  7.94it/s]
23it [00:03,  8.11it/s]
24it [00:03,  8.21it/s]
25it [00:03,  8.22it/s]
26it [00:03,  8.16it/s]
27it [00:03,  8.07it/s]
28it [00:03,  8.04it/s]
29it [00:03,  8.08it/s]
30it [00:03,  7.92it/s]
31it [00:04,  7.82it/s]
32it [00:04,  7.56it/s]
33it [00:04,  7.88it/s]
34it [00:04,  8.07it/s]
35it [00:04,  8.32it/s]
36it [00:04,  8.40it/s]
37it [

<<e : 1>> <<train_loss :  0.01>> <<train_accuracy : 56.00>> <<val_loss :  0.69>> <<val_accuracy : 55.00>>


  5%|████▏                                                                              | 1/20 [00:14<04:33, 14.41s/it]
0it [00:00, ?it/s]
1it [00:00,  8.45it/s]
2it [00:00,  8.37it/s]
3it [00:00,  7.69it/s]
4it [00:00,  7.11it/s]
5it [00:00,  6.40it/s]
6it [00:00,  6.64it/s]
7it [00:01,  6.81it/s]
8it [00:01,  6.99it/s]
9it [00:01,  6.92it/s]
10it [00:01,  6.80it/s]
11it [00:01,  7.07it/s]
12it [00:01,  6.95it/s]
13it [00:01,  6.87it/s]
14it [00:02,  7.19it/s]
15it [00:02,  7.31it/s]
16it [00:02,  7.02it/s]
17it [00:02,  6.96it/s]
18it [00:02,  7.09it/s]
19it [00:02,  7.25it/s]
20it [00:02,  7.54it/s]
21it [00:02,  7.19it/s]
22it [00:03,  7.26it/s]
23it [00:03,  7.50it/s]
24it [00:03,  7.35it/s]
25it [00:03,  7.34it/s]
26it [00:03,  7.32it/s]
27it [00:03,  7.05it/s]
28it [00:03,  7.29it/s]
29it [00:04,  7.27it/s]
30it [00:04,  7.47it/s]
31it [00:04,  7.14it/s]
32it [00:04,  7.47it/s]
33it [00:04,  8.02it/s]
34it [00:04,  8.13it/s]
35it [00:04,  8.35it/s]
36it [00:04,  8.59it/s]
37it [

<<e : 2>> <<train_loss :  0.01>> <<train_accuracy : 56.00>> <<val_loss :  0.69>> <<val_accuracy : 54.00>>


 10%|████████▎                                                                          | 2/20 [00:29<04:21, 14.55s/it]
0it [00:00, ?it/s]
1it [00:00,  8.24it/s]
2it [00:00,  8.26it/s]
3it [00:00,  8.12it/s]
4it [00:00,  8.25it/s]
5it [00:00,  7.90it/s]
6it [00:00,  7.68it/s]
7it [00:00,  7.37it/s]
8it [00:01,  7.67it/s]
9it [00:01,  7.62it/s]
10it [00:01,  7.39it/s]
11it [00:01,  7.58it/s]
12it [00:01,  7.64it/s]
13it [00:01,  7.74it/s]
14it [00:01,  7.60it/s]
15it [00:01,  7.53it/s]
16it [00:02,  7.92it/s]
17it [00:02,  8.05it/s]
18it [00:02,  7.59it/s]
19it [00:02,  7.25it/s]
20it [00:02,  7.01it/s]
21it [00:02,  6.91it/s]
22it [00:02,  7.02it/s]
23it [00:03,  7.23it/s]
24it [00:03,  7.14it/s]
25it [00:03,  7.39it/s]
26it [00:03,  7.44it/s]
27it [00:03,  7.54it/s]
28it [00:03,  7.58it/s]
29it [00:03,  7.54it/s]
30it [00:03,  7.53it/s]
31it [00:04,  7.35it/s]
32it [00:04,  7.07it/s]
33it [00:04,  7.34it/s]
34it [00:04,  7.22it/s]
35it [00:04,  6.78it/s]
36it [00:04,  6.18it/s]
37it [

...

 90%|█████████████████████████████████████████████████████████████████████████▊        | 18/20 [04:14<00:28, 14.07s/it]
0it [00:00, ?it/s]
1it [00:00,  8.04it/s]
2it [00:00,  8.14it/s]
3it [00:00,  8.25it/s]
4it [00:00,  8.17it/s]
5it [00:00,  7.99it/s]
6it [00:00,  7.89it/s]
7it [00:00,  8.40it/s]
8it [00:00,  8.33it/s]
9it [00:01,  8.18it/s]
10it [00:01,  8.51it/s]
11it [00:01,  8.50it/s]
12it [00:01,  8.48it/s]
13it [00:01,  8.52it/s]
14it [00:01,  8.41it/s]
15it [00:01,  8.36it/s]
16it [00:01,  8.65it/s]
17it [00:02,  8.61it/s]
18it [00:02,  8.54it/s]
19it [00:02,  8.54it/s]
20it [00:02,  8.60it/s]
21it [00:02,  8.47it/s]
22it [00:02,  8.46it/s]
23it [00:02,  8.29it/s]
24it [00:02,  8.38it/s]
25it [00:02,  8.24it/s]
26it [00:03,  8.24it/s]
27it [00:03,  8.08it/s]
28it [00:03,  7.82it/s]
29it [00:03,  7.76it/s]
30it [00:03,  7.99it/s]
31it [00:03,  8.01it/s]
32it [00:03,  8.06it/s]
33it [00:03,  8.11it/s]
34it [00:04,  8.25it/s]
35it [00:04,  8.19it/s]
36it [00:04,  8.20it/s]
37it [

<<e : 19>> <<train_loss :  0.01>> <<train_accuracy : 55.00>> <<val_loss :  0.69>> <<val_accuracy : 54.00>>


 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [04:28<00:13, 13.94s/it]
0it [00:00, ?it/s]
1it [00:00,  6.65it/s]
2it [00:00,  6.81it/s]
3it [00:00,  7.06it/s]
4it [00:00,  7.33it/s]
5it [00:00,  6.81it/s]
6it [00:00,  6.96it/s]
7it [00:00,  7.13it/s]
8it [00:01,  7.04it/s]
9it [00:01,  7.58it/s]
10it [00:01,  8.01it/s]
11it [00:01,  7.68it/s]
12it [00:01,  7.73it/s]
13it [00:01,  7.70it/s]
14it [00:01,  7.14it/s]
15it [00:02,  7.03it/s]
16it [00:02,  7.23it/s]
17it [00:02,  7.66it/s]
18it [00:02,  7.56it/s]
19it [00:02,  7.49it/s]
20it [00:02,  7.06it/s]
21it [00:02,  7.05it/s]
22it [00:03,  7.06it/s]
23it [00:03,  7.12it/s]
24it [00:03,  6.90it/s]
25it [00:03,  7.37it/s]
26it [00:03,  7.37it/s]
27it [00:03,  7.31it/s]
28it [00:03,  7.34it/s]
29it [00:04,  6.89it/s]
30it [00:04,  6.87it/s]
31it [00:04,  6.47it/s]
32it [00:04,  6.38it/s]
33it [00:04,  6.69it/s]
34it [00:04,  7.10it/s]
35it [00:04,  7.20it/s]
36it [00:04,  7.45it/s]
37it [

<<e : 20>> <<train_loss :  0.01>> <<train_accuracy : 56.00>> <<val_loss :  0.69>> <<val_accuracy : 54.00>>


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [04:43<00:00, 14.20s/it]
